In [ ]:
#http://www.xicidaili.com/

In [1]:
import requests
import re
import json
import time
from bs4 import BeautifulSoup
from lxml import etree
from requests.exceptions import ReadTimeout,ConnectionError

In [2]:
def cleanString(oldString):
    newString = oldString.replace('\n', '')
    newString = newString.replace('\\', '')
    newString = newString.replace('  ', '')
    newString = newString.replace('   ', '')
    return newString

In [3]:
def saveTotxt(disNum, date_times, user1s, user2s, content_goods, title):
    with open('data/' + title + '.txt', 'a+', encoding='utf-8') as f:
        for i in range(date_times.__len__()):
            f.write(str(disNum) + ',')
            f.write(date_times[i] + ',')
            f.write(user1s[i] + ',')
            f.write(user2s[i] + ',')
            f.write(content_goods[i].replace(',','，') + '\n')
            disNum += 1
        f.close()
    return disNum

In [4]:
def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError:
        return False
    return True

In [5]:
def getDiscussionFromOnePage(url, headers, proxies, disNum, title):
    readTimeout = False
    readTimeout_count = 0
    while True:
        try:
            response = requests.get(url, headers=headers, timeout = 20)
            break
        except ReadTimeout:
            print('Get a timeout')
            readTimeout_count += 1
        except ConnectionError:
            print('Get a timeout')
            readTimeout_count += 1
        if readTimeout_count == 5:
            readTimeout = True
            break
            
    if readTimeout == True:
        print('Always timeout. Shiped current page')
    else:
        if response.status_code == 200 and is_json(response.text):
            jsonData = response.json()
            htmlStr = str(jsonData['data']['html'])
            htmlStr = cleanString(htmlStr)
            html = etree.HTML(htmlStr)
            html_data = html.xpath('//div[@class="list_box"]/div[@class="list_ul"]/div[@class="list_li S_line1 clearfix"]')
            date_times = []
            user1s = []
            user2s = []
            content_goods = []
            for d in html_data:
                date_time = d.xpath('./child::div[@class="list_con"]/div[@class="WB_func clearfix"]/div[@class="WB_from S_txt2"]//text()')[0]
                aNum = d.xpath('./child::div[@class="list_con"]/div[@class="WB_text"]/a').__len__()
                content_bad = d.xpath('./child::div[@class="list_con"]/div[@class="WB_text"]//text()')
                user1 = content_bad[0]
                content_good = ''
                if content_bad.__len__()<4 or aNum<2:
                    for i in range(content_bad.__len__()-1):
                        content_good += content_bad[i+1]
                    user2 = ''
                else:
                    for i in range(content_bad.__len__()-3):
                        content_good += content_bad[i+3]
                    user2 = content_bad[2].replace('@','')
                content_good = content_good.replace('：','')
                content_good = content_good.replace(':','')
                date_times.append(date_time)
                user1s.append(user1)
                user2s.append(user2)
                content_goods.append(content_good)
            return saveTotxt(disNum, date_times, user1s, user2s, content_goods, title)
        elif response.status_code == 414:
            print('Get a 414 on ContentGet')
            time.sleep(30)
            return getDiscussionFromOnePage(url, headers, proxies, disNum, title)
        elif response.status_code == 503:
            print('Get a 503 on ContentGet')
            time.sleep(3)
            return getDiscussionFromOnePage(url, headers, proxies, disNum, title)
        elif (response.status_code == 200 and is_json(response.text) == False) or response.status_code == 404:
            print('Cookies out time')
            exit()
        else:
            print(str(response.status_code))
            print('Unknow ERROR')
            exit()

In [6]:
def getAUrlByPage(old_url, pageNum):
    url_split = old_url.split('&')
    new_url = ''
    for i in range(url_split.__len__()):
        if url_split[i].find('page=') == -1 and i < url_split.__len__()-1:
            new_url += url_split[i] + '&'
        elif url_split[i].find('page=') == -1 and i == url_split.__len__()-1:
            new_url += url_split[i]
        else:
            new_url += 'page=' + str(pageNum) + '&'
    return new_url

In [7]:
def getWeiboDiscussion(url, disNum, title, currPage):
    proxies = {
        "https":"https://61.128.208.94:3128",
        "https":"https://183.129.207.73:14823",
        "https":"https://223.223.187.195:80",
        "https":"https://203.93.209.163:53281",
        "https":"https://59.37.18.243:3128"
    }
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'Cookie': 'SINAGLOBAL=4454129403737.665.1540017698992; wvr=6; wb_view_log_5485650173=1440*9001; YF-Ugrow-G0=169004153682ef91866609488943c77f; SCF=AkAGBzmnRrBsZyLGJUwLkv_HUqu2cIVaKpTGSpJa2wjbmiEvsvr-ESPlL0a6ctHSmsImUu8ZYTdZlX2_ozo9pfU.; SUB=_2A252z47oDeRhGeNK41cX9S7NzD-IHXVVvOcgrDV8PUJbmtBeLRTQkW9NSVAC9kn3nCaaE6CG1ua__mrG69wVuDK7; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhkzAEoOqAfFKxUuP63.RO75JpX5K-hUgL.Fo-X1h-cSK5pS0e2dJLoIfQLxK-L1K5L1heLxK-L1hML1h5LxKML1-zL1KnLxKBLBonLB-BLxKMLBozLB--LxKqL1KqLBo.LxKBLBonLB.2LxKML12zL1KMLxKBLBonLB-BLxKML12-L12zt; SUHB=0Jvg4GJCEcSlcz; ALF=1571631670; SSOLoginState=1540095673; YF-V5-G0=5468b83cd1a503b6427769425908497c; _s_tentry=login.sina.com.cn; UOR=,,login.sina.com.cn; Apache=3309785789590.145.1540095677557; ULV=1540095677594:2:2:1:3309785789590.145.1540095677557:1540017699000; YF-Page-G0=5c7144e56a57a456abed1d1511ad79e8',
        'Host': 'weibo.com',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36'
    }
    pageNumFlag = False
    try:
        requests.adapters.DEFAULT_RETRIES = 5
        response = requests.get(url, headers=headers, timeout = 20)
        pageNumFlag = True
    except ReadTimeout:
        print('Failed. Get page num failed1')
    except ConnectionError:
        print('Failed. Get page num failed2')
    if pageNumFlag == True and response.status_code == 200 and is_json(response.text):
        jsonData = response.json()
        pageNum = int(jsonData['data']['page']['totalpage'])
        print(pageNum)
        for i in range(pageNum-currPage):
            url = getAUrlByPage(url, i+currPage+1)
            disNum = getDiscussionFromOnePage(url, headers, proxies, disNum, title)
            print(str(i+currPage+1) + ' page finished (Total: ' + str(pageNum) + ')')
#         countNum = int(jsonData['data']['count'])
#         countNum += disNum - 1
#         i = 0
#         while(disNum < countNum):
#             url = getAUrlByPage(url, i+currPage+1)
#             disNum = getDiscussionFromOnePage(url, headers, proxies, disNum, title)
#             print(str(disNum) + ' discussion finished (Total: ' + str(countNum) + ')')
#             i += 1
        print('Finished')
    elif pageNumFlag == True and response.status_code == 414:
        print('Get a 414 on pageNumGet')
        time.sleep(30)
        return getWeiboDiscussion(url, disNum, title, currPage)
    elif pageNumFlag == True and response.status_code == 503:
        print('Get a 503 on pageNumGet')
        time.sleep(3)
        return getWeiboDiscussion(url, disNum, title, currPage)
    elif (response.status_code == 200 and is_json(response.text) == False) or response.status_code == 404:
        print('Cookies out time')
        exit()
    else:
        print(str(response.status_code))
        print('Unknow ERROR')
        exit()
    return disNum

In [8]:
title = '范冰冰偷税漏税'
disNum = 12021
currPage = 706
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4291181112267037&root_comment_max_id=603649047522147&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4291110169902050&root_comment_max_id=154620403457&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, 0)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4291017299808168&root_comment_max_id=154608376240&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, 0)

948
707 page finished (Total: 948)
708 page finished (Total: 948)
709 page finished (Total: 948)
710 page finished (Total: 948)
711 page finished (Total: 948)
712 page finished (Total: 948)
713 page finished (Total: 948)
714 page finished (Total: 948)
715 page finished (Total: 948)
716 page finished (Total: 948)
717 page finished (Total: 948)
718 page finished (Total: 948)
719 page finished (Total: 948)
720 page finished (Total: 948)
721 page finished (Total: 948)
722 page finished (Total: 948)
723 page finished (Total: 948)
724 page finished (Total: 948)
725 page finished (Total: 948)
726 page finished (Total: 948)
727 page finished (Total: 948)
728 page finished (Total: 948)
729 page finished (Total: 948)
730 page finished (Total: 948)
731 page finished (Total: 948)
732 page finished (Total: 948)
733 page finished (Total: 948)
734 page finished (Total: 948)
735 page finished (Total: 948)
736 page finished (Total: 948)
737 page finished (Total: 948)
738 page finished (Total: 948)
739 

19 page finished (Total: 275)
20 page finished (Total: 275)
21 page finished (Total: 275)
22 page finished (Total: 275)
23 page finished (Total: 275)
24 page finished (Total: 275)
25 page finished (Total: 275)
26 page finished (Total: 275)
27 page finished (Total: 275)
28 page finished (Total: 275)
29 page finished (Total: 275)
30 page finished (Total: 275)
31 page finished (Total: 275)
32 page finished (Total: 275)
33 page finished (Total: 275)
34 page finished (Total: 275)
35 page finished (Total: 275)
36 page finished (Total: 275)
37 page finished (Total: 275)
38 page finished (Total: 275)
39 page finished (Total: 275)
40 page finished (Total: 275)
41 page finished (Total: 275)
42 page finished (Total: 275)
43 page finished (Total: 275)
44 page finished (Total: 275)
45 page finished (Total: 275)
46 page finished (Total: 275)
47 page finished (Total: 275)
48 page finished (Total: 275)
49 page finished (Total: 275)
50 page finished (Total: 275)
51 page finished (Total: 275)
52 page fi

361
1 page finished (Total: 361)
2 page finished (Total: 361)
3 page finished (Total: 361)
4 page finished (Total: 361)
5 page finished (Total: 361)
6 page finished (Total: 361)
7 page finished (Total: 361)
8 page finished (Total: 361)
9 page finished (Total: 361)
10 page finished (Total: 361)
11 page finished (Total: 361)
12 page finished (Total: 361)
13 page finished (Total: 361)
14 page finished (Total: 361)
15 page finished (Total: 361)
16 page finished (Total: 361)
17 page finished (Total: 361)
18 page finished (Total: 361)
19 page finished (Total: 361)
20 page finished (Total: 361)
21 page finished (Total: 361)
22 page finished (Total: 361)
23 page finished (Total: 361)
24 page finished (Total: 361)
25 page finished (Total: 361)
26 page finished (Total: 361)
27 page finished (Total: 361)
28 page finished (Total: 361)
29 page finished (Total: 361)
30 page finished (Total: 361)
31 page finished (Total: 361)
32 page finished (Total: 361)
33 page finished (Total: 361)
34 page finishe

258 page finished (Total: 361)
259 page finished (Total: 361)
260 page finished (Total: 361)
261 page finished (Total: 361)
262 page finished (Total: 361)
263 page finished (Total: 361)
264 page finished (Total: 361)
265 page finished (Total: 361)
266 page finished (Total: 361)
267 page finished (Total: 361)
268 page finished (Total: 361)
269 page finished (Total: 361)
270 page finished (Total: 361)
271 page finished (Total: 361)
272 page finished (Total: 361)
273 page finished (Total: 361)
274 page finished (Total: 361)
275 page finished (Total: 361)
276 page finished (Total: 361)
277 page finished (Total: 361)
278 page finished (Total: 361)
279 page finished (Total: 361)
280 page finished (Total: 361)
281 page finished (Total: 361)
282 page finished (Total: 361)
283 page finished (Total: 361)
284 page finished (Total: 361)
285 page finished (Total: 361)
286 page finished (Total: 361)
287 page finished (Total: 361)
288 page finished (Total: 361)
289 page finished (Total: 361)
290 page